In [ ]:
"""
gray_cropped_video(Color Coded Lyrics 형식의 비디오에서 한국어 가사에 해당되는 부분만 잘라낸 뒤, gray_scale 처리한 비디오)에 들어있는 영상을 wav.file로 변환한다.
"""
import moviepy.editor as mp
clip = mp.VideoFileClip("./gray_cropped_video/가을 아침/가을 아침.mp4")
clip.audio.write_audiofile("audio.wav")

In [1]:
"""
scene_object_dict이라는 pickle 파일은 PySceneDetect와 EasyOCR을 조합하여, 가사가 바뀌는 장면을 Scene으로 정의하고,
각 Scene은 Scene마다 Scene 시작 시간(start), Scene 끝 시간(end), EasyOCR로 검출된 가사(ocr_lyrics) 등의 정보를 담고 있다.
Example:
    >>> print(scene_object_dict['에필로그']
    {'에필로그': [Scene(singer_name='아이유', song_name='에필로그', start=0.0, end=20.987654320987655, ocr_lyrics='예필로그', corrected_lyrics=None),
    Scene(singer_name='아이유', song_name='에필로그', start=20.987654320987655, end=31.631631631631635, ocr_lyrics='나름 알게 되어서 기뻐는지 나름 사랑해서 좋아엇는지', corrected_lyrics=None),
    Scene(singer_name='아이유', song_name='에필로그', start=31.631631631631635, end=41.70837504170838, ocr_lyrics='우리 위해 불컷던 지나간 노래들이 여전히 위로가 되는지', corrected_lyrics=None),
    ...}
"""
import pickle
with open('./file/pickle/scene_object_dict.pkl', 'rb') as f:
    scene_object_dict = pickle.load(f)

In [13]:
from typing import List
from model.scene import Scene
import re

def remove_short_ocr_lyrics(scene_list:List[Scene])->List[Scene]:
    """OCR로 검출된 가사의 길이가 너무 짧으면 제외"""
    return list(filter(lambda x:x.ocr_lyrics != '' and len(x.ocr_lyrics)>7, scene_list))

def remove_unreadable_text(scene_list:List[Scene])->List[Scene]:
    """한국어로 읽기 힘든 데이터면 제외"""
    for scene in scene_list:
        scene.ocr_lyrics = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", scene.ocr_lyrics)
    return scene_list


In [14]:
ocr_scene_song_dict = {key: remove_short_ocr_lyrics(remove_unreadable_text(value)) for key, value in scene_object_dict.items()}